In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import display

# КРИТИЧНО для PyCharm
%matplotlib inline

import matplotlib.pyplot as plt




plt.rcParams["figure.figsize"] = (10, 4)
plt.rcParams["axes.grid"] = True



In [ ]:
# Ячейка 2 — загрузка и очистка функции
def load_and_clean(path='prodres.csv'):
    df = pd.read_csv(path, encoding='utf-8')

    # Универсальная нормализация "пустоты"
    df = df.replace(
        {
            '–': np.nan,
            '—': np.nan,
            '-': np.nan,
            '': np.nan,
        },
        regex=False
    )

    id_col = df.columns[0]
    role_cols = list(df.columns[1:])

    for c in role_cols:
        df[c] = pd.to_numeric(df[c], errors='coerce')

    return df, id_col, role_cols


df, id_col, role_cols = load_and_clean('prodres.csv')
display(df.head(12))
print("Роли:", role_cols)


In [ ]:
# Ячейка 3 — summary по ролям
def role_summary(df, role_cols):
    rows = []

    for c in role_cols:
        s = df[c]

        row = {
            "participants": s.notna().sum(),
            "zero_scores": (s == 0).sum(),
            "positive_scores": (s > 0).sum(),

            "ge_40": (s >= 40).sum(),
            "ge_60": (s >= 60).sum(),
            "ge_80": (s >= 80).sum(),
            "ge_90": (s >= 90).sum(),

            "participation_pct": round(s.notna().mean() * 100, 2),

            "mean_all": s.mean(),
            "mean_positive": s[s > 0].mean(),

            "median": s.median(),
            "max": s.max(),
        }

        rows.append(pd.Series(row, name=c))

    return pd.DataFrame(rows)



summary_roles = role_summary(df, role_cols)

summary_roles = summary_roles.sort_values(
    by=["positive_scores", "mean_positive"],
    ascending=False
)



display(summary_roles.style.format(precision=2))


In [ ]:
# Ячейка 4 — свод по трекам (строкам)
def per_track_summary(df, id_col, role_cols):
    r = df.copy()

    r['num_non_null'] = r[role_cols].notna().sum(axis=1)
    r['num_positive'] = (r[role_cols] > 0).sum(axis=1)
    r['mean'] = r[role_cols].mean(axis=1)
    r['max'] = r[role_cols].max(axis=1)

    # безопасный idxmax
    best_role = pd.Series(index=r.index, dtype="object")

    mask = r['num_non_null'] > 0
    best_role.loc[mask] = r.loc[mask, role_cols].idxmax(axis=1)

    r['best_role'] = best_role

    return r[[id_col, 'num_non_null', 'num_positive', 'mean', 'max', 'best_role']]

per_track = per_track_summary(df, id_col, role_cols)
display(per_track)
per_track.to_csv('prodres_artifacts/per_track_summary.csv', encoding='utf-8-sig', index=False)
print("Per-track summary saved to prodres_artifacts/per_track_summary.csv")

per_track = per_track_summary(df, id_col, role_cols)

display(
    per_track
    .sort_values(by="max", ascending=False)
    .head(50)
    .style
    .format({"mean": "{:.2f}", "max": "{:.2f}"})
)



In [ ]:
# Ячейка 5 — top-k (например top-3)
def top_k_by_role(df, role_cols, k=3, id_col='Шифр'):
    tops = {}
    for c in role_cols:
        tmp = df[[id_col, c]].dropna().sort_values(by=c, ascending=False).head(k).reset_index(drop=True)
        tops[c] = tmp
    # Convert to one combined table
    combined = []
    for role, table in tops.items():
        for _, row in table.iterrows():
            combined.append({'role': role, id_col: row[id_col], 'value': row[role]})
    tops_df = pd.DataFrame(combined)
    return tops, tops_df

tops, tops_df = top_k_by_role(df, role_cols, k=3, id_col=id_col)
display(tops_df)
tops_df.to_csv('prodres_artifacts/topk_by_role.csv', encoding='utf-8-sig', index=False)
print("Top-K saved to prodres_artifacts/topk_by_role.csv")

tops, tops_df = top_k_by_role(df, role_cols, k=3, id_col=id_col)

for role, table in tops.items():
    if table.empty:
        continue
    print(f"\n=== {role} ===")
    display(table.style.format(precision=2))



In [ ]:
# Ячейка 6 — пороговая аналитика
def threshold_overview(df, role_cols, thresholds=(40,60,80,90)):
    overview = {}
    overview['tracks_total'] = len(df)
    overview['tracks_any_positive'] = (df[role_cols] > 0).any(axis=1).sum()
    for t in thresholds:
        overview[f'tracks_>={t}_any_field'] = (df[role_cols] >= t).any(axis=1).sum()
        # сколько треков имеют >=t в N или более полях
        for n in range(1, len(role_cols)+1):
            overview.setdefault(f'ge{t}_in_at_least_{n}', (df[role_cols] >= t).sum(axis=1).ge(n).sum())
    # сочетания: сколько треков >=t одновременно в конкретных парах (пример: пары ролей)
    pairs = []
    for i in range(len(role_cols)):
        for j in range(i+1, len(role_cols)):
            a,b = role_cols[i], role_cols[j]
            counts = {}
            for t in thresholds:
                counts[f'ge{t}'] = ((df[a] >= t) & (df[b] >= t)).sum()
            pairs.append({'pair': f'{a} & {b}', **counts})
    pairs_df = pd.DataFrame(pairs)
    return overview, pairs_df

overview, pairs_df = threshold_overview(df, role_cols)
print("Overview (ключевые):")
for k,v in overview.items():
    if isinstance(v, (int, float)):
        print(f"- {k}: {v}")
print("\nПары ролей с подсчетом треков, которые >= порога одновременно:")
display(pairs_df.head(20))
pairs_df.to_csv('prodres_artifacts/pairs_thresholds.csv', encoding='utf-8-sig', index=False)

overview, pairs_df = threshold_overview(df, role_cols)

display(pd.DataFrame.from_dict(overview, orient="index", columns=["value"]))

display(
    pairs_df
    .sort_values(by="ge80", ascending=False)
    .head(20)
)



In [ ]:
# Ячейка 7 — корреляция
def correlations(df, role_cols):
    pearson = df[role_cols].corr(method='pearson')
    spearman = df[role_cols].corr(method='spearman')
    return pearson, spearman

pearson, spearman = correlations(df, role_cols)
print("Pearson:")
display(pearson)
print("Spearman:")
display(spearman)

# Сохраним heatmap (matplotlib)
plt.figure(figsize=(10,8))
plt.imshow(pearson, aspect='auto', cmap='viridis')
plt.colorbar()
plt.xticks(range(len(pearson.columns)), pearson.columns, rotation=90)
plt.yticks(range(len(pearson.index)), pearson.index)
plt.title('Pearson correlation matrix')
plt.tight_layout()
plt.savefig('prodres_artifacts/pearson_corr.png', bbox_inches='tight')
plt.close()
print("Pearson heatmap saved to prodres_artifacts/pearson_corr.png")

pearson, spearman = correlations(df, role_cols)

print("Pearson")
display(pearson.round(2))

plt.figure(figsize=(10, 6))
plt.imshow(pearson, aspect="auto")
plt.colorbar()
plt.xticks(range(len(pearson.columns)), pearson.columns, rotation=90)
plt.yticks(range(len(pearson.index)), pearson.index)
plt.title("Pearson correlation")
plt.tight_layout()
plt.show()  # ← ВАЖНО



In [ ]:
# Ячейка 8 — визуализации
def plot_histograms(df, role_cols, outdir='prodres_artifacts'):
    for c in role_cols:
        s = df[c].dropna()
        if len(s) == 0:
            continue
        plt.figure(figsize=(6,3))
        plt.hist(s, bins='auto')
        plt.title(f'Distribution — {c}')
        plt.xlabel('score')
        plt.ylabel('count')
        fname = f'{outdir}/hist_{c.replace(" ", "_").replace("-", "")}.png'
        plt.tight_layout()
        plt.savefig(fname)
        plt.close()

plot_histograms(df, role_cols)

for c in role_cols:
    s = df[c].dropna()
    if s.empty:
        continue

    plt.figure()
    plt.hist(s, bins="auto")
    plt.title(f"Distribution — {c}")
    plt.xlabel("score")
    plt.ylabel("count")
    plt.show()

# Bar chart: count >= thresholds per role
thresholds = [40,60,80,90]
counts_roles = {t: (df[role_cols] >= t).sum() for t in thresholds}
counts_df = pd.DataFrame(counts_roles)
counts_df.plot(kind='bar', figsize=(12,4))
plt.title('Counts of scores >= thresholds per role')
plt.ylabel('count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('prodres_artifacts/threshold_counts_per_role.png')
plt.close()
print("Гистограммы и bar-chart сохранены в prodres_artifacts/")

thresholds = [40, 60, 80, 90]
counts_df = pd.DataFrame(
    {t: (df[role_cols] >= t).sum() for t in thresholds}
)

display(counts_df)

counts_df.plot(kind="bar")
plt.title("Scores >= threshold per role")
plt.ylabel("count")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()



In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


# Ячейка 9 — PCA + KMeans
def pca_and_cluster(df, role_cols, n_clusters=3):
    # Берём только строки/роли с хотя бы одним числом (или можно fillna=0)
    X = df[role_cols].fillna(0).values  # замена NaN на 0 — одна стратегия
    scaler = StandardScaler()
    Xs = scaler.fit_transform(X)
    pca = PCA(n_components=2)
    Xp = pca.fit_transform(Xs)
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(Xp)
    res = pd.DataFrame(Xp, columns=['PC1','PC2'], index=df.index)
    res['cluster'] = labels
    return res, pca, kmeans, scaler

if PCA is not None:
    pca_res, pca_model, kmeans_model, scaler = pca_and_cluster(df, role_cols, n_clusters=3)
    pca_plot = pca_res.copy()
    pca_plot['id'] = df[id_col].values
    # matplotlib scatter
    plt.figure(figsize=(7,5))
    for cl in sorted(pca_plot['cluster'].unique()):
        sub = pca_plot[pca_plot['cluster']==cl]
        plt.scatter(sub['PC1'], sub['PC2'], label=f'cluster {cl}', alpha=0.8)
    for i,row in pca_plot.iterrows():
        plt.text(row['PC1']+0.01, row['PC2']+0.01, str(row['id']), fontsize=8)
    plt.legend()
    plt.title('PCA 2D + KMeans clusters')
    plt.tight_layout()
    plt.savefig('prodres_artifacts/pca_kmeans.png')
    plt.close()
    display(pca_res)
    print("PCA + KMeans plot saved to prodres_artifacts/pca_kmeans.png")
else:
    print("scikit-learn не доступен — установи sklearn чтобы выполнить PCA/кластеризацию.")

pca_res, _, _, _ = pca_and_cluster(df, role_cols, n_clusters=3)

display(pca_res)

plt.figure(figsize=(7, 5))
for cl in sorted(pca_res["cluster"].unique()):
    sub = pca_res[pca_res["cluster"] == cl]
    plt.scatter(sub["PC1"], sub["PC2"], label=f"cluster {cl}")

plt.legend()
plt.title("PCA + KMeans")
plt.show()


In [ ]:
# Ячейка 10 — интерактивная панель для порогов (опционально)
try:
    import ipywidgets as widgets
    from IPython.display import clear_output
    have_widgets = True
except Exception:
    have_widgets = False

if have_widgets:
    role_select = widgets.SelectMultiple(options=role_cols, description='Роли', rows=6)
    threshold_slider = widgets.IntSlider(value=40, min=0, max=100, step=1, description='Порог')
    run_button = widgets.Button(description='Показать')

    out = widgets.Output()
    display(widgets.HBox([role_select, threshold_slider, run_button]), out)

    def on_run(b):
        with out:
            clear_output()
            sel = list(role_select.value)
            t = threshold_slider.value
            if len(sel)==0:
                print("Выбери хотя бы 1 роль.")
                return
            subset = df[[id_col]+sel]
            mask = (subset[sel] >= t).any(axis=1)
            display(subset[mask])
            print(f"Найдено {mask.sum()} треков с >= {t} в выбранных ролях.")
    run_button.on_click(on_run)
else:
    print("ipywidgets не установлен — установи ipywidgets для интерактивности.")


In [ ]:
# Ячейка 11 — экспорт основных артефактов и краткий PDF (при необходимости)
# Сохраняем очищенный CSV
df.to_csv('prodres_artifacts/prodres_clean.csv', index=False, encoding='utf-8-sig')
summary_roles.to_csv('prodres_artifacts/summary_roles.csv', encoding='utf-8-sig')
per_track.to_csv('prodres_artifacts/per_track_summary.csv', index=False, encoding='utf-8-sig')
tops_df.to_csv('prodres_artifacts/topk_by_role.csv', index=False, encoding='utf-8-sig')

print("Все ключевые CSV сохранены в prodres_artifacts/.")
print("Если хочешь — могу добавить генерацию PDF-отчёта (нужен reportlab / matplotlib).")
